In [ ]:
#tensor
from tensorflow import keras
from tensorflow.keras import layers

#keras
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.utils import to_categorical


import numpy as np
import h5py
from pathlib import Path
import matplotlib.pyplot as plt

from keras import backend as keras_backend

#y finance stuff
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# ACTUALLY THE BEST
from stock_module4 import download_and_save_stock_data
symbol = "AAPL"
days = 10000
output_csv_path = "stock_data.csv"

download_and_save_stock_data(symbol, days, output_csv_path)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from datetime import datetime, timedelta

# Assuming you already have a DataFrame with stock data named stock_data
stock_data = pd.read_csv('stock_data.csv')
# Sort the DataFrame by date
stock_data['Date'] = pd.to_datetime(stock_data['Date'])
stock_data.sort_values(by='Date', inplace=True)
stock_data.reset_index(drop=True, inplace=True)

# Define the prediction timeframes
one_day = 1
one_month = 30
one_year = 365

# Create a function to prepare the data for LSTM
def prepare_data(df, target_col, look_back):
    data = df[target_col].values
    data = data.reshape(-1, 1)
    scaler = MinMaxScaler()
    data = scaler.fit_transform(data)
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i+look_back])
        y.append(data[i+look_back])
    X, y = np.array(X), np.array(y)
    return X, y, scaler

# Split the data into train and test sets
test_size = one_year
look_back = one_year
X, y, scaler = prepare_data(stock_data[['Close']], 'Close', look_back)
X_train, y_train = X[:-test_size], y[:-test_size]
X_test, y_test = X[-test_size:], y[-test_size:]

# Build the LSTM model

model = Sequential()
model.add(LSTM(units=10, recurrent_dropout = 0.2, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Define callbacks
callbacks = [
    ModelCheckpoint('best_model.h6', save_best_only=True),
    EarlyStopping(patience=10, restore_best_weights=True)
]

# Train the model
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), callbacks=callbacks)

Do not know if below works

In [ ]:
# Load the best model
model = keras.models.load_model('best_model.h6')

# Predict stock prices
last_date = stock_data['Date'].max()
next_day = last_date + timedelta(days=one_day)
next_month = last_date + timedelta(days=one_month)
next_year = last_date + timedelta(days=one_year)

In [ ]:
# ...
# Prepare input data for predictions
input_data = stock_data[['Close']].tail(look_back).values
input_data = input_data.reshape(-1, 1)  # Reshape to 2D
scaled_input = scaler.transform(input_data)

# Predict one day, one month, and one year ahead
predicted_price_next_day = model.predict(scaled_input)
predicted_price_next_month = model.predict(scaled_input)
predicted_price_next_year = model.predict(scaled_input)

# Inverse transform the predicted prices to get actual prices
predicted_price_next_day = scaler.inverse_transform(predicted_price_next_day)
predicted_price_next_month = scaler.inverse_transform(predicted_price_next_month)
predicted_price_next_year = scaler.inverse_transform(predicted_price_next_year)

print(f"Predicted Price for the next day ({next_day}): {predicted_price_next_day[0][0]}")
print(f"Predicted Price for the next month ({next_month}): {predicted_price_next_month[0][0]}")
print(f"Predicted Price for the next year ({next_year}): {predicted_price_next_year[0][0]}")


In [ ]:
# Evaluate the model on test data
test_loss = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {test_loss}")

# Make predictions on the test data
y_pred = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred)
y_test = scaler.inverse_transform(y_test)

# Calculate evaluation metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Visualize the predictions
plt.figure(figsize=(12, 6))
plt.plot(stock_data['Date'].tail(len(y_test)), y_test, label='Actual Prices', color='blue')
plt.plot(stock_data['Date'].tail(len(y_test)), y_pred, label='Predicted Prices', color='red')
plt.title('Stock Price Prediction - Test Data')
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.legend()
plt.show()